In [4]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/EPFL/AppliedDataScience/Project/ada-2022-project-adlucere2022/src/"

Mounted at /content/drive


Imports

In [29]:
# Basic Stuff
import pandas as pd
import re
import string

from collections import Counter

# NLP
import spacy
from nltk.stem import WordNetLemmatizer
import gensim
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Load list of stopwords

In [22]:
sp = spacy.load('en_core_web_sm')
spacy_stopwords = sp.Defaults.stop_words
stopwords_dict = Counter(spacy_stopwords)

In [23]:

lemmatizer =WordNetLemmatizer()
#cleaning "pipeline"  
def clean_summary(text, stopwords, lemmatization):

      
      #perform casefolding
      text =text.casefold()

      #remove punctuations for each twitter
      text = ' '.join(text_ for text_ in text.split() if text_ not in string.punctuation)

      #remove multiply commas and dots everywhere in tweets      
      text = re.sub('\.|,*','', text)
      
      if stopwords:
          #remove the stopwords
          text = ' '.join([word for word in text.split() if word not in stopwords_dict])         
            
      if lemmatization :
          #perform lemmatization
          text = ' '.join(lemmatizer.lemmatize(text_)  for text_ in text.split() )

      #remove the tokens length less than 2 again if some may appear after the above preprocessing
      text = ' '.join(text_ for text_ in text.split() if len(text_)>2)
      
      return  text


In [50]:
plots = pd.read_csv(path + '../data/movie_data_imdbscores_final.csv')[['movie_id', 'plot_summary']]

In [51]:
# Lemmatize and filter stopwords ->  gensim simple preprocessing pipeline -> join into strings
plots.plot_summary = plots.plot_summary.apply(lambda x : clean_summary(x,stopwords=True, lemmatization=True))\
                          .apply(lambda x : gensim.utils.simple_preprocess(str(x), deacc=True))\
                          .apply(lambda x : " ".join(x))

In [52]:
plots.head()

,movie_id,plot_summary
0,975900,set second half nd century film depicts mar pl...
1,9363483,series murder rich young woman arizona bear di...
2,261236,eva upper class housewife frustrated leaf arro...
3,18998739,year evil morgana return claim fingall talisma...
4,12621957,merlin apprentice named maxim horvath betrays ...


In [53]:
plots.to_csv(path + '../data/plot/plot_summaries_preprocessed.csv', index = False)